# Advent of Code 2016, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202016%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [324]:
⎕IO←1
]box on -style=max -trains=tree -fns=on
]rows on -fold=3

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→────────────────┐
│Was OFF -fold=off│
└─────────────────┘

### Day 1: No Time for a Taxicab
https://adventofcode.com/2016/day/1

Given a set of relative directions ("turn right, travel 45 units"), what is the final [Manhattan](https://en.wikipedia.org/wiki/Taxicab_geometry) distance from origin?

There are many different ways to solve this. Here we simply trace out the full path, converting relative directions to absolute positions, and check the last point for the answer.

We can convert from relative to absolute direction using a simple vector `1 2 3 4` for north, east, south, west respectively. Turning right corresponds to a vector rotation of 1, and turning left a rotation of ¯1.

Key insight is that we need to "connect the dots" between the points referred to in the input data, which only contains the turning points.

In [91]:
DAY1←,{(1↑⍵),⍎1↓⍵}¨⎕CSV'data/2016/01.txt'     ⍝ Convert strings to numbers

In [109]:
]dinput
ExpandPath←{
    pos←0 0 ⋄ dir←1 2 3 4
    {⊃,/,⊆¨⍵}⍣≡{                              ⍝ Flatten returned list
        (rl mag)←⍵
        dir {⍵='R':(1⊖dir)⋄¯1⊖dir}←rl         ⍝ Rotate direction vector 
        unitv←dir[1]⊃(0 ¯1)(1 0)(0 1)(¯1 0)   ⍝ Find corresponding unit vector in new direction
        points←pos∘+¨↓(⍳mag)∘.×unitv          ⍝ Expand the "gaps"
        pos+←mag×unitv                        ⍝ Update the turning point
        points
    }¨⍵
}

Our start state is `dir←1 2 3 4 ⋄ pos←0 0`, which is "north-facing" and our origin. For the answer we're only interested in the final position, and the Manhattan distance is simply the sum of the absolute values of x and y of the final position.

In [111]:
Day1p1←{+/|⊃¯1↑ExpandPath ⍵}     ⍝ Sum absolute values of x and y for last position
Day1p1 DAY1 ⍝ 239        

239

For part 2 we need to find the first position visited twice. We can use the built-in [key](http://help.dyalog.com/17.1/index.htm#Language/Primitive%20Operators/Key.htm?Highlight=%E2%8C%B8) function (`⌸`) to build a frequency table, which will be ordered in the same way as the data.

In [121]:
Day1p2←{firstRepeat←1⌷⍸{⍵[2]=2}¨↓{⍺,≢⍵}⌸path←ExpandPath ⍵⋄+/⊃|path[firstRepeat]}

In [122]:
Day1p2 DAY1 ⍝ 141

141

### Day 2: Bathroom Security
https://adventofcode.com/2016/day/2

Create a matrix where the row number is the starting digit, and cols 1-4 corresponding to the values for a move URDL respectively. We can then map the letters URDL to numbers 1 2 3 and 4 respectively.

Parts 1 and 2 are basically identical, bar the initial setup of this matrix.

In [376]:
DAY2←⊃⎕NGET'data/2016/02.txt'1
KEYMAP←9 4 ⍴ 1 2 4 1 2 3 5 1 3 3 6 2 1 5 7 4 2 6 8 4 3 6 9 5 4 8 7 7 5 9 8 7 6 9 9 8

In [377]:
Day2←{⍺←5⋄0=≢⍵:1↓⍺⋄(⍺,((⊃¯1↑⍺) {⍵≡⍬:⍺⋄KEYMAP[⍺;1⊃⍵]∇1↓⍵} 'URDL'⍳1⊃⍵))∇1↓⍵}

In [383]:
Day2 DAY2 ⍝ 35749

┌→────────┐
│3 5 7 4 9│
└~────────┘

For part 2 we have a different shape keypad, and a few more keys: A, B, C and D. We map those to 10-13 and tediously construct a new key map (by hand).

In [186]:
KEYMAP←13 4 ⍴ 1 1 3 1 2 3 6 2 1 4 7 2 4 4 8 3 5 6 5 5 2 7 10 5 3 8 11 6 4 9 12 7 9 9 9 8 6 11 10 10 7 12 13 10 8 12 12 11 11 13 13 13

In [187]:
('ABCD',⍨⍳9)[Day2 DAY2] ⍝ 9365C

┌→────────┐
│9 3 6 5 C│
└+────────┘

### Day 3: Squares With Three Sides
https://adventofcode.com/2016/day/3

Not much to do here apart from massaging the input array into the right shape.

In [199]:
DAY3←⍎¨⊃⎕NGET'data/2016/03.txt'1

In [202]:
Valid←{(a b c)←⍵⋄((a+b)>c)∧((a+c)>b)∧((c+b)>a)}
+/Valid¨DAY3 ⍝ 1050

1050

In [207]:
PART2←⍉↑DAY3
+/Valid¨↓(3÷⍨≢∊PART2) 3 ⍴ ∊PART2 ⍝ 1921

1921

### Day 4: Security Through Obscurity
https://adventofcode.com/2016/day/4

A set of strings, consisting of letters and hyphens (the "encrypted name"), followed by digits (the "sector id"), followed by more letters enclosed in square brackets (the "checksum").

A string is valid iff the checksum is the five most common letters in the encrypted name, in order, with ties broken by alphabetization.

In [213]:
RegexGroups←{⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⊢ ⍵}
DAY4←'^([^\d]+)(\d+)\[([^\]]+)\]$' RegexGroups ⊃⎕NGET'data/2016/04.txt'1

To validate a given string, we do the following:

1. Remove all hyphens
2. Create a letter histogram mapping each letter to its frequency
3. Sort the histogram in descending order on frequency and separate keys and values into vectors
4. Partition the keys vector based on neighbouring letters with the same frequencies
5. Sort each partition alphabetically, and flatten to single character vector
6. If the first 5 letters match the checksum, return the sector id (converted to number).
7. Else return 0.

In [240]:
]dinput
Day4p1←{
    (keys vals)←↓⍉{⍵[⍒⍵[;2];]}{⍺,≢⍵}⌸(~∘'-')1⊃⍵  ⍝ Remove hyphens, make frq table ordered by descending frq
    p←5↑∊{(⊂⍋⍵)⌷⍵}¨keys⊂⍨1,2≠/vals               ⍝ Partition, sort, merge, pick first 5
    p≡3⊃⍵:⍎2⊃⍵⋄0
}

In [241]:
+/Day4p1¨DAY4 ⍝ 245102

245102

For part 2 we are asked to 'decrypt' the string, meaning a circular shift of each letter by the sector id. We can achieve this easily using the dyadic form of [⊖](http://help.dyalog.com/17.1/index.htm#Language/Symbols/Circle%20Bar.htm?Highlight=%E2%8A%96) and the system function [⎕A](http://help.dyalog.com/17.1/index.htm#Language/System%20Functions/a.htm) which holds the upper case letters. 

1. Convert string to upper-case
2. For each letter, rotate ⎕A until the letter is first
3. Rotate by the sector id

For the answer, we're searching for 'NORTHPOLE' amongst the decrypted strings. We can achieve this succinctly using a sequence of maps. However, would be more efficient by stopping after the target string was found.

The odd-looking [819⌶](http://help.dyalog.com/17.1/index.htm#Language/Primitive%20Operators/Case%20Convert.htm?Highlight=819%E2%8C%B6) is a system function to convert a string to uppercase.

In [242]:
Day4p2←{R←⍎2⊃⍵⋄∊{⍵='-':' '⋄1↑(R+¯1+⎕A⍳⍵)⊖⎕A}¨1(819⌶)1⊃⍵}

In [243]:
⍎2⊃DAY4⊃⍨⊃⍸0≠⊃¨'NORTHPOLE'∘(⍸⍷)¨Day4p2¨DAY4 ⍝ 324

324

### Day 5: How About a Nice Game of Chess?
https://adventofcode.com/2016/day/5

Skipping this, as needs MD5 which isn't available for Dyalog on a Mac :(

### Day 6: Signals and Noise
https://adventofcode.com/2016/day/6

Transpose input array. For each row, create a letter histogram, sort based on descending (part 1) and ascending (part 2) frequency, and pick the first letter.

In [266]:
DAY6←↓⍉↑⊃⎕NGET'data/2016/06.txt'1
∊{1 1⌷⍵[⍒⍵[;2];]}¨{⍺,≢⍵}⌸¨DAY6 ⍝ Part 1: qrqlznrl
∊{1 1⌷⍵[⍋⍵[;2];]}¨{⍺,≢⍵}⌸¨DAY6 ⍝ Part 2: kgzdfaon

┌→───────┐
│qrqlznrl│
└────────┘

┌→───────┐
│kgzdfaon│
└────────┘

### Day 7: Internet Protocol Version 7
https://adventofcode.com/2016/day/7

Some medium-level regexing required. 

In [272]:
DAY7←⊃⎕NGET'data/2016/07.txt'1
RegexGroups←{⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⊢ ⍵}

In [273]:
]dinput
Abba←{
    matches←'(.)(.)\2\1'⎕S'&'⊢⍵
    0=≢matches:0
    invalid←≢'(.)\1\1\1'⎕S'&'⊢matches
    0≠invalid-⍨≢matches
}

In [274]:
Day7p1←{(Abba ⍵)∧~Abba '\[.+?\]'⎕S'&'⊢⍵}  ⍝ ABBAs in body, and no ABBAs in hypernet sections

In [275]:
+/Day7p1¨DAY7 ⍝ 115

115

For part 2, in the aba captures, we place the capture groups in an unanchored positive lookahead in order to see any overlapping matches, like in the example given: `zazbz[bzb]cdb`.

In [276]:
]dinput
Day7p2←{
    hypernets←∊'\[.+?\]'⎕S'&'⊢⍵
    rest←'\[.+?\]'⎕R''⊢⍵
    aba←'(?=(.)(.)\1)' RegexGroups rest    
    0=≢aba:0
    bab←∊{(∊⍵)[2 1 2]⎕S'&'⊢hypernets}¨aba
    (≢bab)>0
}

In [277]:
+/Day7p2¨DAY7 ⍝ 231

231

### Day 8: Two-Factor Authentication
https://adventofcode.com/2016/day/8

Some more array-wrangling -- APL home court advantage. The coordinates given are 0-based, so we switch to using `⎕IO←0` to avoid arithmetic. We can create the three functions referred to (rect, row, col) and treat the input data as a ready to go program. Note that the abuse of [⍎](http://help.dyalog.com/17.1/index.htm#Language/Symbols/Execute%20Symbol.htm?Highlight=%E2%8D%8E) here (in the absense of Scheme-y macros) is both unsafe and unsavoury.

In [334]:
⎕IO←0
DISPLAY←6 50⍴0
rect←{(cols rows)←⍵ ⋄ 1@(⊂⍤0,⍳rows cols)⊢⍺}
row←{(A B)←⍵⋄((-B)⊖⍺[A;])@A⊢⍺}
col←{⍉(⍉⍺)row⍵}

The `Compile` function converts the input rows into strings containing valid APL in terms of the three functions `rect`, `row` and `col`.

In [335]:
]dinput
Compile←{ 
    row←⊃'rotate row y=(\d+) by (\d+)'⎕R'⍺ row \1 \2'¨↓⍵
    col←'rotate column x=(\d+) by (\d+)'⎕R'⍺ col \1 \2'¨row
    'rect (\d+)x(\d+)'⎕R'⍺ rect \1 \2'¨col
}

In [336]:
DAY8←Compile⊃⎕NGET'data/2016/08.txt'1

In [337]:
Day8←{0=≢⍵:⍺⋄(⍎0⊃⍵)∇1↓⍵} ⍝ Run the 'program', top to bottom

In [338]:
+/∊DISPLAY Day8 DAY8  ⍝ Part 1: 121

121

The display shows a message! Let's highlight it a bit.

In [346]:
(' '@⊢)0=DISPLAY Day8 DAY8 ⍝ Part 2: RURUCEOEIL

┌→─────────────────────────────────────────────────────────────────────────────────────────────────┐
↓0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 │
└+─────────────────────────────────────────────────────────────────────────────────────────────────┘

### Day 9: Explosives in Cyberspace
https://adventofcode.com/2016/day/9

Decompress simple runlength-encoding scheme. Can be succinctly formulated in terms of recursion. 

In [356]:
⎕IO←1
DAY9←⊃⊃⎕NGET'data/2016/09.txt'1

A run-length marker is two ints separated by an "x" and enclosed in brackets, e.g. (8x2). Let's write a utility function that gets these numbers out (assuming the beginning of the string), and also tacks on the end index -- where the string starts again after the marker. We use `⎕VFI` -- [Verify and Fix Input](http://help.dyalog.com/17.1/index.htm#Language/System%20Functions/vfi.htm?Highlight=%E2%8E%95VFI) -- to convert strings to numbers. 

In [367]:
Marker←{(2⊃'x'⎕VFI 1↓¯1↓end↑⍵),end←⍵⍳')'} 

The decompressed length is 1 for each non-compressed character plus the length×count of any encoded sequence.

In [368]:
]dinput
Day9p1←{
    0=≢⍵:⍺
    '('≠⊃1↑⍵:(⍺+1)∇1↓⍵               ⍝ Uncompressed character -- add 1 and move on
    (length count end)←Marker ⍵
    (⍺+length×count)∇(end+length)↓⍵  ⍝ Add length×count and skip the marker and the length when carrying on
}

In [369]:
0 Day9p1 DAY9 ⍝ 74532

74532

For part 2 we need [big int support](http://dfns.dyalog.com/n_big.htm), which we can find in the 'dfns' workspace.

In [370]:
'big'⎕CY'dfns'

Simiar to part 1, but we need an extra recursive step to expand already expanded chunks. Note that this means we're no longer tail-recursive, but the recursive depth is pretty modest.

In [373]:
]dinput
Day9p2←{
    0=≢⍵:⍺
    '('≠⊃1↑⍵:(⍺ +big 1)∇1↓⍵
    (length count end)←Marker ⍵
    total←count ×big 0∇length↑end↓⍵  ⍝ Find length of expansion, including any embedded markers
    (⍺ +big total)∇(end+length)↓⍵    ⍝ Next, as per part 1
}

In [375]:
0 Day9p2 DAY9 ⍝ 11558231665

┌→──────────┐
│11558231665│
└───────────┘

### Day 10: Balance Bots
https://adventofcode.com/2016/day/10

Note: as the bots are 0-indexed, we use 0-based indexing.

This turned out a bit messier than I had hoped.

In [506]:
⎕IO←0   
DAY10←⊃⎕NGET'data/2016/10.txt'1
RegexGroups←{⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)}⊢⍵}

We need to do some massaging to turn the input data into a matrix we can use. Our matrix will consist of rows where the row number is the bot index, and the fields:

    type low-target type high-target low high
    
The `type` fields are booleans where 0 represents another bot, and 1 represents the output register.

In [507]:
]dinput
MakeBots←{
    bots←↑'bot (\d+) gives low to (output|bot) (\d+) and high to (output|bot) (\d+)' RegexGroups ⍵
    bots[;0]←⍎¨bots[;0]      ⍝ Col 0 to numbers
    bots[;1]←(∪⍳⊢)bots[;1]   ⍝ Col 1: 'bot' -> 0, 'output' -> 1
    bots[;2]←⍎¨bots[;2]      ⍝ Col 2 to numbers
    bots[;3]←(∪⍳⊢)bots[;3]   ⍝ Col 3: 'bot' -> 0, 'output' -> 1
    bots[;4]←⍎¨bots[;4]      ⍝ Col 2 to numbers
    
    bots←(({⍵[⍋⍵[;0];]}bots)[;1 2 3 4],0),0  ⍝ Order by bot-id, drop first col, add two 0 cols for inputs
    
    ⍝ Set the known values
    (vals rows)←↓⍉↑⍎¨¨'value (\d+) goes to bot (\d+)' RegexGroups ⍵
    bots←vals@(rows,¨4)⊢bots    ⍝ Pretend no dupes
    dupe←⊃{({1≠≢⍵}⌸⍵)⌿∪⍵}rows   ⍝ Assume single dupe
    bots[dupe;5]←vals[⊃1↑dupe(⍸⍷)rows]
    bots (0 0 0) ⍝ Tack on the output register state
}

The main guts is to distribute the low and high value to other bots (or output registers, part 2) according to the spec. We can use the same function for the low and high transfer as they only differ in which index to look at.

In [508]:
]dinput
Give←{
    Output←{(1⊃⍵)>2:⍺⋄(0⊃⍵)@(1⊃⍵)⊢⍺}
    (bots outv)←⍺ ⋄ (bid offset)←⍵             ⍝ Offest: 0 = low, 1 = high
    bot←bid⌷bots
    val←bots[bid;4+offset]
    tid←bot[1+2×offset] 
    bots[bid;4+offset]←0                       ⍝ Zero the value we're giving away
    bot[2×offset]=1:bots (outv Output val tid)
    vals←(tid⌷bots)[4 5],bot[4+offset] 
    bots[tid;4 5]←1↓vals[⍋vals]                ⍝ Maintain ordering
    bots outv
}

We can only run bots which have two values, so find those, and apply their instruction sets.

In [509]:
]dinput
Run←{
    Ready←{⍸~(0=⍵[;4])∨0=⍵[;5]}
    ⍵ {
        0=≢⍵:⍺ 
        (⍺{(⍺ Give ⍵ 0)Give ⍵ 1}0⊃⍵)∇ 1↓⍵
    } Ready 0⊃⍵
}

For part 1, our stopping criterion is that a bot has low=17 and high=61.

In [512]:
Day10p1←{bots←0⊃⍵⋄(⊃⍸17=bots[;4])=⊃⍸61=bots[;5]}

In [513]:
(BOTS OUTV)←Run⍣Day10p1⊢MakeBots DAY10
⊃⍸17=BOTS[;4] ⍝ 54

54

For part 2 we stop once we've got data in the first three slots of the output register.

In [516]:
Day10p2←{3=+/0∘<1⊃⍵}

In [517]:
(BOTS OUTV)←Run⍣Day10p2⊢MakeBots DAY10
×/OUTV ⍝ 7847

7847

### Day 11: Radioisotope Thermoelectric Generators
https://adventofcode.com/2016/day/11

This problem is somewhat involved to solve for the general case with an A* heuristic search, but let's not go there. The problem is actually solvable with a pen and paper if we make a few observations -- firstly, we're only asked to count the number of steps required for a solution, not actually show any of the actual path steps required. Secondly, by some pen and paper noodling, it can be shown that to move n items up one level requires 

    2×(n-1)-1
    
steps. Assuming there is a solution, we can count the number of steps required considering only the _number of elements_, rather than the elements themselves. Instead of generating all possible states, ordering them in terms of fitness etc, we assume that we pick only the optimal state, as we know the cost. Simply empty one floor at a time, moving upwards. 

In [518]:
⎕IO←1

In [524]:
]dinput
Day11←{
    moves←0 ⋄ elems←+/⍵
    _←{
        lowest←⊃1↑⍸0∘<⍵ ⍝ Find first non-empty floor
        moves+←¯1+2×¯1+lowest⌷⍵
        0@lowest⊢⍵[lowest]∘+@(1+lowest)⊢⍵
    }⍣{elems=⊃¯1↑⍺}⊢⍵
    moves
}

In [525]:
Day11 5 1 4 0 ⍝ Part 1: 33
Day11 8 5 1 0 ⍝ Part 2: 61

33

61

### Day 12: Leonardo's Monorail
https://adventofcode.com/2016/day/12

Not much to this -- essentially the same as https://adventofcode.com/2015/day/23. Part 2 is a bit slow, as the input code has some deliberate crude loops that are easily refactored out if we wanted to.

In [20]:
⎕IO←1
DAY12←↑{6::⍵⋄⍎⍵}¨¨' '(≠⊆⊢)¨⊃⎕NGET'data/2016/12.txt'1 ⍝ Split on space and convert numbers

In [16]:
]dinput
Day12←{
    mem←⍵
    ri←{'abcd'⍳⍵}                               ⍝ Map register name to index
    val←{(1=2|⎕DR)⍵:⍵⋄⍺[ri ⍵]}                  ⍝ If ⍵ is an int literal, return it. Otherwise reg lookup
    cpy←{(x y)←⍵⋄1∘+@5⊢(⍺ val x)@(ri y)⊢⍺}
    jnz←{(x y)←⍵⋄(0>⍨⍺ val x):(⍺ val y)∘+@5⊢⍺⋄1∘+@5⊢⍺}
    inc←{x←ri ⊃⍵⋄1∘+@5⊢1∘+@x⊢⍺}
    dec←{x←ri ⊃⍵⋄1∘+@5⊢¯1∘+@x⊢⍺}
    {
        ip←¯1↑⍵
        instr←ip⌷mem⋄op←⊃instr
        ⍵ (⍎op) 1↓instr         
    }⍣{(¯1↑⍺)>≢mem}⍺
}

In [18]:
1↑0 0 0 0 1 Day12 DAY12 ⍝ Part 1: 318003

318003

In [19]:
1↑0 0 1 0 1 Day12 DAY12 ⍝ Part 2: 9227657 -- note: this takes several minutes to run

9227657

### Day 13: A Maze of Twisty Little Cubicles
https://adventofcode.com/2016/day/13

Shortest path in a graph. The graph is defined in terms of a neighbourhood function. Once this is formulated, the path can be found using [Dijkstra's shortest path algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm), which is a useful tool to have in the Advent of Code toolbox. The `heapq` namespace can be found in the same repo.

In [99]:
⎕IO←1

In [29]:
]LINK.Create heapq src/heapq

Already linked: #.graph → /Users/stefan/work/dyalog/AoCDyalog/src/graph

Already linked: #.heapq → /Users/stefan/work/dyalog/AoCDyalog/src/heapq

Each location is classified as either "floor" or "wall". We encode this as the function `F`:

In [87]:
F←{(⍺<0)∨⍵<0:0⋄0=2|+/(2∘⊥⍣¯1)1364+(⍺×⍺)+(3×⍺)+⍵+(2×⍺×⍵)+⍵×⍵}     ⍝ The magic function given in the problem

The neighbourhood function gives the valid subset of the 4-connected neighbours of a point:

In [101]:
Neighbours←{(x y)←⊃⍵⋄(F/↑n)/n←(⊂x y)+(¯1 0) (1 0) (0 ¯1) (0 1)}

Dijkstra's shortest path algorithm is a form of best-first search. It considers the neighbourhood of the current node, recording best so far total path lengths. Our version assumes that graph edges are unweighed, meaning that the cost of traversing an edge is uniform at 1. By using a heap queue we ensure that we prioritise lower-cost paths.

In [235]:
]dinput
Day13p1←{                   ⍝ Naive Dijkstra's shortest path search, uniform cost 1
    start←⍺⋄end←⍵
    costKeys←1500⌶,start
    costVals←,0
    Cost←{⍵∊costKeys:costVals[⊃costKeys⍳⍵] ⋄ ⌊/⍬}
    cameFromKeys←1500⌶,start
    cameFromVals←,start

    _←{
        (frontier queueItem)←##.heapq.Pop ⍵
        newCost←1+Cost (current←2⊃⊃queueItem)
        current≡end:⍬
        valid←{newCost<Cost⊂⍵}¨neighbours←Neighbours current
        frontier {
            0=≢⍵:⍺
            vertex←1↑⍵
            queue←⍺ ##.heapq.Push ⊂newCost(vertex)
            cameFromKeys,←vertex
            cameFromVals,←current
            vertex∊costKeys:queue∇(1↓⍵)⊣costVals[⊃costKeys⍳vertex]←newCost
            costKeys,←vertex ⋄ costVals,←newCost
            queue∇1↓⍵   
        } valid/neighbours
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push⊂0 start

    ⍝ Reconstruct the path
    path←⍬
    _←{path,←⍵⋄cameFromVals[⊃cameFromKeys⍳⍵]}⍣{start≡⍺} end
    ⊖path
}

In [123]:
≢(⊂1 1) Day13p1 ⊂31 39

86

For part 2, we want to find the number of distinct locations that can be reached in maximum 50 steps. For this we can simplify our above Dijkstra implementation in that we no longer need to keep track of the path itself.

In [108]:
]dinput
Day13p2←{
    costKeys←1500⌶,⍵                                           ⍝ Make this array a hash table                            
    costVals←,0
    Cost←{⍵∊costKeys:costVals[⊃costKeys⍳⍵] ⋄ ⌊/⍬}              ⍝ Default cost is +infinity
    _←{
        (frontier queueItem)←##.heapq.Pop ⍵
        newCost←1+Cost (current←2⊃⊃queueItem)    
        newCost>50:frontier                                    ⍝ Cut-off at 50 steps
        valid←{newCost<Cost⊂⍵}¨neighbours←Neighbours current   ⍝ Find neighbours either new or at lower cost
        frontier {
            0=≢⍵:⍺
            vertex←1↑⍵
            queue←⍺ ##.heapq.Push ⊂newCost(vertex)
            vertex∊costKeys:queue∇(1↓⍵)⊣costVals[⊃costKeys⍳vertex]←newCost  ⍝ Old, but lower-cost
            costKeys,←vertex ⋄ costVals,←newCost                            ⍝ New vertex
            queue∇1↓⍵   
        } valid/neighbours
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push⊂0 ⍵                     ⍝ Repeat until queue is empty
    ≢costKeys
}

In [109]:
Day13p2 ⊂1 1 ⍝ 127

127

### Day 14: One-Time Pad
https://adventofcode.com/2016/day/14

Skipping this, as no native MD5 on Dyalog for MacOS :(

### Day 15: Timing is Everything
https://adventofcode.com/2016/day/15

Back to the regular schedule of one-liners.

In [232]:
⎕IO←0

This shrank down nicely. The key parts are:

    {⍵|⍵-⍳≢⍵}          ⍝ Given the disk sizes, calculate the end state - the first digit of every disk we need
    SIZES {⍵⊖⍳⍺}¨STATE ⍝ Rotate the disks to their given start states
    1∘⊖¨              ⍝ "Tick" all disks by one position
    
Basically it's `Tick⍣{first digits matches end state}`.

In [233]:
Day15←{e←{⍵|⍵-⍳≢⍵}⍺⋄c←¯1⋄_←1∘⊖¨⍣{c+←1⋄e≡⊃¨⍺}⊢⍺{⍵⊖⍳⍺}¨⍵⋄c}

In [234]:
17 7 19 5 3 13    Day15 1 0 2 0 0 5   ⍝ Part 1: 317371
17 7 19 5 3 13 11 Day15 1 0 2 0 0 5 0 ⍝ Part 2: 2080951

317371

2080951

### Day 16: Dragon Checksum
https://adventofcode.com/2016/day/16

[Dragon](https://en.wikipedia.org/wiki/Dragon_curve) curve.

In [1]:
⎕IO←1
Dragon←{⍵,0,~⊖⍵}

Rebuilding user command cache... done

In [2]:
Checksum←{=/(2÷⍨≢⍵)2⍴⍵}

In [3]:
Day16←{size←⍺⋄Checksum⍣{2|≢⍺} size↑Dragon⍣{(≢⍺)≥size} ⍵}

In [6]:
272      Day16 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 ⍝ Part 1: 11100110111101110
35651584 Day16 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 ⍝ Part 2: 10001101010000101

1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0

1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1